# 🌿 AgriTech - Plant Disease Detection Demo

This notebook demonstrates the **Plant Disease Detection** model from the AgriTech project.

It classifies plant leaf images into disease categories using a **CNN (Convolutional Neural Network)**.

Supports:
- Tomato diseases (38 classes from PlantVillage dataset)
- General plant disease detection
- Healthy vs diseased classification

---
**GitHub:** https://github.com/omroy07/AgriTech

## 📦 Step 1: Install Dependencies

In [ ]:
!pip install tensorflow numpy pillow matplotlib requests opencv-python

## 📚 Step 2: Import Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import json
import requests
from io import BytesIO
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

print('✅ Core libraries imported!')

# TensorFlow import (optional - only needed for direct model inference)
try:
    import tensorflow as tf
    print(f'✅ TensorFlow {tf.__version__} available!')
    TF_AVAILABLE = True
except ImportError:
    print('⚠️ TensorFlow not installed. You can still use the API method.')
    TF_AVAILABLE = False

## 🏷️ Step 3: Disease Class Labels

In [ ]:
# 38 class labels from PlantVillage dataset (used by AgriTech disease model)
CLASS_NAMES = [
    'Apple___Apple_scab',
    'Apple___Black_rot',
    'Apple___Cedar_apple_rust',
    'Apple___healthy',
    'Blueberry___healthy',
    'Cherry_(including_sour)___Powdery_mildew',
    'Cherry_(including_sour)___healthy',
    'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
    'Corn_(maize)___Common_rust_',
    'Corn_(maize)___Northern_Leaf_Blight',
    'Corn_(maize)___healthy',
    'Grape___Black_rot',
    'Grape___Esca_(Black_Measles)',
    'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
    'Grape___healthy',
    'Orange___Haunglongbing_(Citrus_greening)',
    'Peach___Bacterial_spot',
    'Peach___healthy',
    'Pepper,_bell___Bacterial_spot',
    'Pepper,_bell___healthy',
    'Potato___Early_blight',
    'Potato___Late_blight',
    'Potato___healthy',
    'Raspberry___healthy',
    'Soybean___healthy',
    'Squash___Powdery_mildew',
    'Strawberry___Leaf_scorch',
    'Strawberry___healthy',
    'Tomato___Bacterial_spot',
    'Tomato___Early_blight',
    'Tomato___Late_blight',
    'Tomato___Leaf_Mold',
    'Tomato___Septoria_leaf_spot',
    'Tomato___Spider_mites Two-spotted_spider_mite',
    'Tomato___Target_Spot',
    'Tomato___Tomato_Yellow_Leaf_Curl_Virus',
    'Tomato___Tomato_mosaic_virus',
    'Tomato___healthy'
]

# Disease treatment database
DISEASE_TREATMENTS = {
    'Apple___Apple_scab':            'Apply fungicides containing captan or mancozeb. Remove infected leaves.',
    'Apple___Black_rot':             'Prune infected branches. Apply copper-based fungicide during dormant season.',
    'Apple___Cedar_apple_rust':      'Apply myclobutanil or mancozeb. Remove nearby juniper trees.',
    'Corn_(maize)___Common_rust_':   'Apply triazole or strobilurin fungicides early. Use resistant varieties.',
    'Potato___Early_blight':         'Apply chlorothalonil or mancozeb. Ensure proper crop rotation.',
    'Potato___Late_blight':          'Apply metalaxyl + mancozeb. Remove infected plants immediately.',
    'Tomato___Early_blight':         'Remove infected leaves. Apply copper-based fungicide or chlorothalonil.',
    'Tomato___Late_blight':          'Apply mefenoxam or metalaxyl. Destroy infected plants completely.',
    'Tomato___Bacterial_spot':       'Use copper bactericide sprays. Avoid overhead irrigation.',
    'Tomato___Leaf_Mold':            'Improve air circulation. Apply chlorothalonil or copper fungicide.',
    'Tomato___Septoria_leaf_spot':   'Apply mancozeb or chlorothalonil. Mulch around base of plants.',
    'Tomato___Target_Spot':          'Apply azoxystrobin or difenoconazole. Remove heavily infected leaves.',
    'Tomato___Tomato_Yellow_Leaf_Curl_Virus': 'No cure - remove infected plants. Control whitefly vectors.',
    'Grape___Black_rot':             'Apply mancozeb or captan. Remove mummified fruit from previous year.',
    'Pepper,_bell___Bacterial_spot': 'Use copper sprays. Avoid overhead watering. Use resistant varieties.'
}

print(f'✅ Loaded {len(CLASS_NAMES)} disease classes')
print(f'   Healthy classes: {sum(1 for c in CLASS_NAMES if "healthy" in c)}')
print(f'   Disease classes: {sum(1 for c in CLASS_NAMES if "healthy" not in c)}')

## 🔌 Step 4: Option A - Use the AgriTech API

In [ ]:
BASE_URL = 'http://localhost:5000'

def predict_disease_api(image_path):
    """
    Send an image to the AgriTech disease detection API.
    Requires the Flask server to be running: python app.py
    """
    try:
        with open(image_path, 'rb') as img_file:
            files = {'file': img_file}
            response = requests.post(f'{BASE_URL}/predict_disease', files=files, timeout=30)
            response.raise_for_status()
            return response.json()
    except requests.exceptions.ConnectionError:
        return {'error': 'Server not running. Start with: python app.py'}
    except FileNotFoundError:
        return {'error': f'Image not found: {image_path}'}
    except Exception as e:
        return {'error': str(e)}

# Test with local image
result = predict_disease_api('sample_disease_image.jpg')
print('API Result:', json.dumps(result, indent=2))

## 🧠 Step 5: Option B - Load Model Directly (TensorFlow)

In [ ]:
IMG_SIZE = (224, 224)  # Standard input size for the CNN

def preprocess_image(image_path_or_pil, img_size=IMG_SIZE):
    """
    Load and preprocess a plant leaf image for model inference.
    
    Preprocessing steps:
    1. Resize to model input size (224x224)
    2. Convert to RGB
    3. Normalize pixel values to [0, 1]
    4. Add batch dimension
    """
    if isinstance(image_path_or_pil, str):
        img = Image.open(image_path_or_pil).convert('RGB')
    else:
        img = image_path_or_pil.convert('RGB')

    img = img.resize(img_size)
    img_array = np.array(img) / 255.0           # Normalize
    img_array = np.expand_dims(img_array, axis=0) # Add batch dim
    return img_array


if TF_AVAILABLE:
    model_paths = [
        '../Plant Disease Detection/plant_disease_model.h5',
        '../tomato disease detection/model.h5',
        '../Disease prediction/model.h5',
        'model.h5'
    ]

    disease_model = None
    for path in model_paths:
        if os.path.exists(path):
            disease_model = tf.keras.models.load_model(path)
            print(f'✅ Disease model loaded from: {path}')
            print(f'   Input shape: {disease_model.input_shape}')
            break

    if disease_model is None:
        print('⚠️ No pre-trained model found.')
        print('   Paths checked:')
        for p in model_paths:
            print(f'     - {p}')
        print('\n   To use this notebook:')
        print('   1. Train the model using the Disease prediction folder')
        print('   2. Or use Option A (API method) with the running Flask server')
else:
    print('⚠️ TensorFlow not available. Use Option A (API) instead.')
    disease_model = None

## 🔍 Step 6: Run Disease Detection on an Image

In [ ]:
def detect_disease(image_path, model, class_names, top_k=3):
    """
    Detect plant disease from a leaf image.

    Parameters:
        image_path  : path to the leaf image file
        model       : loaded TensorFlow/Keras model
        class_names : list of disease class labels
        top_k       : number of top predictions to return

    Returns:
        dict: {
            'disease': str,
            'confidence': float,
            'is_healthy': bool,
            'top_predictions': list,
            'treatment': str
        }
    """
    img_array = preprocess_image(image_path)
    predictions = model.predict(img_array, verbose=0)
    pred_probs = predictions[0]

    # Top K predictions
    top_indices = np.argsort(pred_probs)[::-1][:top_k]
    top_preds = [
        {'class': class_names[i], 'confidence': round(float(pred_probs[i]) * 100, 2)}
        for i in top_indices
    ]

    best_class = class_names[top_indices[0]]
    is_healthy = 'healthy' in best_class.lower()
    treatment  = DISEASE_TREATMENTS.get(best_class, 'Consult an agricultural expert for treatment.')

    return {
        'disease':         best_class,
        'confidence':      top_preds[0]['confidence'],
        'is_healthy':      is_healthy,
        'top_predictions': top_preds,
        'treatment':       treatment if not is_healthy else 'Plant is healthy! No treatment needed.'
    }


# --- Run detection (requires model to be loaded) ---
if disease_model and os.path.exists('sample_disease_image.jpg'):
    result = detect_disease('sample_disease_image.jpg', disease_model, CLASS_NAMES)

    print('=' * 55)
    print('       🌿 DISEASE DETECTION RESULT')
    print('=' * 55)
    status = '✅ HEALTHY' if result['is_healthy'] else '⚠️ DISEASED'
    print(f'  Status       : {status}')
    print(f'  Diagnosis    : {result["disease"]}')
    print(f'  Confidence   : {result["confidence"]}%')
    print(f'  Treatment    : {result["treatment"]}')
    print('\n  Top 3 Predictions:')
    for i, pred in enumerate(result['top_predictions'], 1):
        print(f'    {i}. {pred["class"]} ({pred["confidence"]}%)')
    print('=' * 55)
elif not disease_model:
    print('⚠️ Model not loaded. Please load the model in Step 5.')
else:
    print('⚠️ sample_disease_image.jpg not found in examples folder.')

## 🖼️ Step 7: Visualize Prediction with Image

In [ ]:
def visualize_prediction(image_path, result):
    """
    Show the image alongside prediction results.
    """
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))

    # Left: Leaf image
    img = Image.open(image_path)
    axes[0].imshow(img)
    axes[0].set_title('📷 Input Leaf Image', fontsize=13)
    axes[0].axis('off')

    # Right: Top-3 prediction bar chart
    labels = [p['class'].replace('___', '\n') for p in result['top_predictions']]
    confs  = [p['confidence'] for p in result['top_predictions']]
    colors = ['#2E8B57' if 'healthy' in l else '#CC3300' for l in labels]

    bars = axes[1].barh(labels, confs, color=colors)
    axes[1].set_xlabel('Confidence (%)')
    axes[1].set_title('🔬 Top Predictions', fontsize=13)
    axes[1].set_xlim(0, 100)

    for bar, val in zip(bars, confs):
        axes[1].text(val + 1, bar.get_y() + bar.get_height()/2,
                     f'{val:.1f}%', va='center', fontsize=10)

    plt.suptitle(f'AgriTech Disease Detection | Status: {"✅ Healthy" if result["is_healthy"] else "⚠️ Diseased"}',
                 fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.savefig('disease_detection_result.png', dpi=150, bbox_inches='tight')
    plt.show()
    print('📊 Result saved as disease_detection_result.png')


if disease_model and os.path.exists('sample_disease_image.jpg'):
    result = detect_disease('sample_disease_image.jpg', disease_model, CLASS_NAMES)
    visualize_prediction('sample_disease_image.jpg', result)
else:
    print('⚠️ Need both a loaded model and sample_disease_image.jpg to visualize.')

## 🧪 Step 8: Test with Your Own Image!

In [ ]:
# ✏️ Change this path to your own leaf image
MY_IMAGE_PATH = 'sample_disease_image.jpg'  # Replace with your image path

if disease_model and os.path.exists(MY_IMAGE_PATH):
    my_result = detect_disease(MY_IMAGE_PATH, disease_model, CLASS_NAMES)

    print('\n🌿 YOUR PLANT ANALYSIS')
    print('=' * 50)
    print(f'  Disease      : {my_result["disease"]}')
    print(f'  Confidence   : {my_result["confidence"]}%')
    print(f'  Healthy?     : {"Yes ✅" if my_result["is_healthy"] else "No ⚠️"}')
    print(f'  Treatment    : {my_result["treatment"]}')
    print('=' * 50)
elif not disease_model:
    print('⚠️ Load the model in Step 5 first.')
else:
    print(f'⚠️ Image not found: {MY_IMAGE_PATH}')